# Importing necessary libraries

In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [48]:
#To ignore warnings

import warnings
warnings.filterwarnings("ignore")


In [49]:
pd.set_option('display.max_columns' , None)
pd.set_option('display.max_rows' , None)
pd.set_option('display.width' , None)

# Import dataset

In [50]:
from sklearn.datasets import load_boston
from sklearn import datasets

df=datasets.load_boston()


In [51]:
newdf=pd.DataFrame(df.data)
newdf.columns=df.feature_names
newdf['MEDV']=df.target
newdf.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


# EDA

In [52]:
newdf.shape

(506, 14)

In [53]:
newdf.describe()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.613524,11.363636,11.136779,0.069170,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,356.674032,12.653063,22.532806
std,8.601545,23.322453,6.860353,0.253994,0.115878,0.702617,28.148861,2.105710,8.707259,168.537116,2.164946,91.294864,7.141062,9.197104
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,0.320000,1.730000,5.000000
25%,0.082045,0.000000,5.190000,0.000000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,375.377500,6.950000,17.025000
50%,0.256510,0.000000,9.690000,0.000000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,391.440000,11.360000,21.200000
75%,3.677083,12.500000,18.100000,0.000000,0.624000,6.623500,94.075000,5.188425,24.000000,666.000000,20.200000,396.225000,16.955000,25.000000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,37.970000,50.000000


# Missing value check

In [54]:
newdf.isnull().sum()

CRIM       0
ZN         0
INDUS      0
CHAS       0
NOX        0
RM         0
AGE        0
DIS        0
RAD        0
TAX        0
PTRATIO    0
B          0
LSTAT      0
MEDV       0
dtype: int64

# Feature selection

## Method 1 - Filter method

In [55]:
cor = newdf.corr()

#Correlation with output variable
cor_target = abs(cor["MEDV"])
#Selecting highly correlated features
relevant_features = cor_target[cor_target>0.5]
relevant_features

RM         0.695360
PTRATIO    0.507787
LSTAT      0.737663
MEDV       1.000000
Name: MEDV, dtype: float64

In [56]:
print(newdf[["LSTAT","PTRATIO"]].corr())
print(newdf[["RM","LSTAT"]].corr())

            LSTAT   PTRATIO
LSTAT    1.000000  0.374044
PTRATIO  0.374044  1.000000
             RM     LSTAT
RM     1.000000 -0.613808
LSTAT -0.613808  1.000000


In [57]:
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif

vif_df = pd.DataFrame() #empty df
vif_df["features"] = X.columns  
vif_df["VIF Factor"] = [vif(X.values, i) for i in range(X.shape[1])]
vif_df.round(2)

,features,VIF Factor
0,CRIM,2.10
1,ZN,2.84
2,INDUS,14.49
3,CHAS,1.15
4,NOX,73.89
5,RM,77.95
6,AGE,21.39
7,DIS,14.70
8,RAD,15.17
9,TAX,61.23


Here we can see variables PTRATIO AND LSTAT have high correlation with output variable then we will select them

## Create X and Y

In [58]:
X = newdf[['PTRATIO' , 'LSTAT']]
Y = newdf['MEDV']



## Model build and prediction

In [59]:
from sklearn.model_selection import train_test_split

#Split the data into test and train

X_train ,X_test, Y_train, Y_test = train_test_split(X, Y , test_size =0.2 , random_state =10)


In [60]:
from sklearn.linear_model import LinearRegression
#create a model object
lm = LinearRegression()
#train the model object
lm.fit(X_train, Y_train)

#print intercept and coefficients
print(lm.intercept_)
print(lm.coef_)

51.50916173884028
[-1.02248562 -0.81909221]


In [61]:
#predit using the model
Y_pred = lm.predict(X_test)
#print (Y_pred)

In [62]:
new_df1 = pd.DataFrame()
new_df1 = X_test

new_df1['actual '] = Y_test
new_df1['predicted '] = Y_pred
#new_df1

## Accuracy

In [63]:
from sklearn.metrics import r2_score,mean_squared_error
import numpy as np

r2=r2_score(Y_test,Y_pred)
print(r2)

rmse=np.sqrt(mean_squared_error(Y_test,Y_pred))
print(rmse)

adjusted_r_squared = 1 - (1-r2)*(len(Y)-1)/(len(Y)-X.shape[1]-1)
print(adjusted_r_squared)


0.5411659385914211
6.9271355586729575
0.539341548685224


## Method 2 - Wrapper method - Back elimination

In [64]:
X = newdf.drop("MEDV" , 1)   #Feature Matrix
y = newdf["MEDV"]

In [65]:
#Adding constant column of ones, mandatory for sm.OLS model
import statsmodels.api as sm
X_1 = sm.add_constant(X)
#Fitting sm.OLS model
model = sm.OLS(y,X_1).fit()
model.pvalues

const      3.283438e-12
CRIM       1.086810e-03
ZN         7.781097e-04
INDUS      7.382881e-01
CHAS       1.925030e-03
NOX        4.245644e-06
RM         1.979441e-18
AGE        9.582293e-01
DIS        6.013491e-13
RAD        5.070529e-06
TAX        1.111637e-03
PTRATIO    1.308835e-12
B          5.728592e-04
LSTAT      7.776912e-23
dtype: float64

In [66]:
cols = list(X.columns)
pmax = 1
while (len(cols)>0):
    p= []
    X_1 = X[cols]
    X_1 = sm.add_constant(X_1)
    model = sm.OLS(y,X_1).fit()
    p = pd.Series(model.pvalues.values[1:],index = cols)      
    pmax = max(p)
    feature_with_p_max = p.idxmax()
    if(pmax>0.05):
        cols.remove(feature_with_p_max)
    else:
        break
selected_features_BE = cols
print(selected_features_BE)

['CRIM', 'ZN', 'CHAS', 'NOX', 'RM', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']


Here we have selected variables - ['CRIM', 'ZN', 'CHAS', 'NOX', 'RM', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']

## Create X and Y

In [67]:
#create X and Y

X1 = newdf[['CRIM', 'ZN', 'CHAS', 'NOX', 'RM', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']]
Y1 = newdf['MEDV']


In [68]:
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif

vif_df = pd.DataFrame() #empty df
vif_df["features"] = X1.columns  
vif_df["VIF Factor"] = [vif(X1.values, i) for i in range(X1.shape[1])]
vif_df.round(2)

,features,VIF Factor
0,CRIM,2.10
1,ZN,2.77
2,CHAS,1.14
3,NOX,62.99
4,RM,71.84
5,DIS,12.03
6,RAD,13.98
7,TAX,49.60
8,PTRATIO,83.69
9,B,20.02


## Model build and prediction

In [69]:
from sklearn.model_selection import train_test_split

#Split the data into test and train

X_train ,X_test, Y_train, Y_test = train_test_split(X1, Y1 , test_size =0.2 , random_state =10)

In [70]:
from sklearn.linear_model import LinearRegression
#create a model object
lm = LinearRegression()
#train the model object
lm.fit(X_train, Y_train)

print(lm.intercept_)
print(lm.coef_)

36.11554990286534
[-1.27574499e-01  6.14043219e-02  2.04292675e+00 -1.51776779e+01
  3.20798331e+00 -1.54649476e+00  3.00284990e-01 -1.21381071e-02
 -8.09111225e-01  1.15929479e-02 -5.61076692e-01]


In [71]:
#predit using the model
Y_pred = lm.predict(X_test)
#print (Y_pred)

## Accuracy

In [72]:
from sklearn.metrics import r2_score,mean_squared_error
import numpy as np

r2=r2_score(Y_test,Y_pred)
print(r2)

rmse=np.sqrt(mean_squared_error(Y_test,Y_pred))
print(rmse)

adjusted_r_squared = 1 - (1-r2)*(len(Y1)-1)/(len(Y1)-X1.shape[1]-1)
print(adjusted_r_squared)

#adjusted r2 is less than r2

0.6763654557506361
5.817726453926353
0.6691590185305085


## Method 3 - Wrapper method - Recursive Feature elimination

In [73]:
X = newdf.drop("MEDV" , 1)   #Feature Matrix
y = newdf["MEDV"]

In [74]:
#RFE (Recursive Feature Elimination)
'''
It then gives the ranking of all the variables, 
1 being most important. It also gives its support, 
True being relevant feature and False being irrelevant feature
'''
from sklearn.feature_selection import RFE
model = LinearRegression()
#Initializing RFE model
rfe = RFE(model, 7)
#Transforming data using RFE
X_rfe = rfe.fit_transform(X,y)  
#Fitting the data to model
model.fit(X_rfe,y)
print(rfe.support_)
print(rfe.ranking_)

[False False False  True  True  True False  True  True False  True False
  True]
[2 4 3 1 1 1 7 1 1 5 1 6 1]


Finding optimal number of variables

In [75]:
#no of features
nof_list=np.arange(1,13)            
high_score=0
#Variable to store the optimum features
nof=0           
score_list =[]
for n in range(len(nof_list)):
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 0)
    model = LinearRegression()
    rfe = RFE(model,nof_list[n])
    X_train_rfe = rfe.fit_transform(X_train,y_train)
    X_test_rfe = rfe.transform(X_test)
    model.fit(X_train_rfe,y_train)
    score = model.score(X_test_rfe,y_test)
    score_list.append(score)
    if(score>high_score):
        high_score = score
        nof = nof_list[n]
print("Optimum number of features: %d" %nof)
print("Score with %d features: %f" % (nof, high_score))

Optimum number of features: 10
Score with 10 features: 0.663581


Re run with value 10

In [76]:
cols = list(X.columns)
model = LinearRegression()
#Initializing RFE model
rfe = RFE(model, 10)             
#Transforming data using RFE
X_rfe = rfe.fit_transform(X,y)  
#Fitting the data to model
model.fit(X_rfe,y)              
temp = pd.Series(rfe.support_,index = cols)
selected_features_rfe = temp[temp==True].index
print(selected_features_rfe)

Index(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'DIS', 'RAD', 'PTRATIO',
       'LSTAT'],
      dtype='object')


 Variables selected - 'CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'DIS', 'RAD', 'PTRATIO', 'LSTAT'

In [77]:
X1 = newdf[['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'DIS', 'RAD', 'PTRATIO','LSTAT']]
Y1 = newdf['MEDV']


In [78]:
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif

vif_df = pd.DataFrame() #empty df
vif_df["features"] = X1.columns  
vif_df["VIF Factor"] = [vif(X1.values, i) for i in range(X1.shape[1])]
vif_df.round(2)

,features,VIF Factor
0,CRIM,2.08
1,ZN,2.66
2,INDUS,11.71
3,CHAS,1.13
4,NOX,65.74
5,RM,72.53
6,DIS,12.82
7,RAD,5.16
8,PTRATIO,75.80
9,LSTAT,9.83


## Scaling

In [79]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaler.fit(X1) #will llearn about X

X1 = scaler.transform(X1) # will transform
#print(X)
#fit_transform available 

## Model build and prediction

In [80]:
from sklearn.model_selection import train_test_split

#Split the data into test and train

X_train ,X_test, Y_train, Y_test = train_test_split(X1, Y1 , test_size =0.2 , random_state =10)

In [81]:
from sklearn.linear_model import LinearRegression
#create a model object
lm = LinearRegression()
#train the model object
lm.fit(X_train, Y_train)

#print intercept and coefficients
print(lm.intercept_)
print(list(zip(X.columns , lm.coef_)))


#zip use to add 2 list

22.373341672046415
[('CRIM', -1.205728234230574), ('ZN', 1.257853616083877), ('INDUS', -0.7348612959415626), ('CHAS', 0.6779974211045794), ('NOX', -2.010189063291302), ('RM', 2.1620469984144175), ('AGE', -3.4149673001335583), ('DIS', 0.8624623626070762), ('RAD', -1.7537304725927942), ('TAX', -4.183425504003439)]


In [82]:
#predit using the model
Y_pred = lm.predict(X_test)
#print (Y_pred)

## Accuracy

In [83]:
from sklearn.metrics import r2_score,mean_squared_error
import numpy as np

r2=r2_score(Y_test,Y_pred)
print(r2)

rmse=np.sqrt(mean_squared_error(Y_test,Y_pred))
print(rmse)

adjusted_r_squared = 1 - (1-r2)*(len(Y1)-1)/(len(Y1)-X1.shape[1]-1)
print(adjusted_r_squared)

#adjusted r2 is less than r2

0.6814855236082098
5.771523247119746
0.675050887721507
